In [1]:
import networkx as nx, math, sys, numpy as np, matplotlib.pyplot as plt, time, csv, os
from tqdm import tqdm
from collections import defaultdict

In [2]:
phi = 0.75
gRed = 0.15
gBlue = 0.15

In [3]:
def read_community(filename):
    with open(filename, "r") as file:
        lines = file.readlines()

    # Extract the number of communities
    num_communities = int(lines[0].strip())

    # Process the data for each node
    for line in lines[1:]:
        node, community = map(int, line.strip().split())
        if community == 1:
            red.append(node)
        elif community == 0:
            blue.append(node)

In [4]:
def makeGraph(filename):
    with open(filename, "r") as file:
        num_nodes = int(file.readline().strip())  # Extract the number of nodes

        for line in file:
            line = line.strip()
            line = line.split("\t")
            source = int(line[0])
            target = int(line[1])
            G.add_edge(source, target)
        for node in G.nodes:
            if G.has_edge(node, node):
                G.remove_edge(node, node)
    #             print(node)

In [5]:
def outRB():
    for node in G.nodes:
        G.nodes[node]['outR'] = 0
        G.nodes[node]['outB'] = 0
        G.nodes[node]['outRList'] = []
        G.nodes[node]['outBList'] = []
        for n in G.neighbors(node):
            if n in red:
                G.nodes[node]['outR'] += 1
                G.nodes[node]['outRList'].append(n)
            elif n in blue:
                G.nodes[node]['outB'] += 1
                G.nodes[node]['outBList'].append(n)

In [6]:
def initlR():
    tmplR = []
    for node in G.nodes:
        if G.out_degree(node) != 0 and G.nodes[node]['outR'] / G.out_degree(node) < phi and G.nodes[node]['outR'] != 0:
            tmplR.append(node)
    return tmplR

In [7]:
def updateD(node):
    for i in G.nodes[node]['outRList']:
#         D[node][i] = (phi - (G.nodes[node]['outR'] / G.degree(node))) / G.nodes[node]['outR']
        D[node][i] = (phi/G.nodes[node]['outR']) - (1/G.out_degree(node))
    for i in G.nodes[node]['outBList']:
#         D[node][i] = ((G.nodes[node]['outR'] / G.degree(node)) - phi) / G.nodes[node]['outB']
        D[node][i] = ((1-phi)/G.nodes[node]['outB']) - (1/G.out_degree(node))

In [8]:
def cmpFormula():
    tmpGains = {}
    pRlist = np.inner(Q, color)
    for node in G.nodes:
        G.nodes[node]['pR'] = pRlist[node]
    
    for node in lR:
        g = gamma[node][node]
        sPkR = 0
        sPkB = 0
        sPkxR = 0
        sPkxB = 0
        fTerm = g/((1-g)*(phi - (G.nodes[node]['outR'] / G.out_degree(node))))
    
        dR = 1/G.nodes[node]['outR']
        dB = 1/G.nodes[node]['outB']

        for n in G.nodes[node]['outRList']:
            sPkR += G.nodes[n]['pR']
            sPkxR += Q[n][node]
        for n in G.nodes[node]['outBList']:
            sPkB += G.nodes[n]['pR']
            sPkxB += Q[n][node]

        numerator = dR*sPkR - dB*sPkB
        denominator = fTerm-(dR*sPkxR - dB*sPkxB)
        tmpGains[node] = pr[node]*numerator/denominator
    return tmpGains

In [9]:
graphFile = '/out_graph.txt'
G = nx.DiGraph()
red = []
blue = []
read_community(sys.path[0] + '/out_community.txt')
nodesList = sorted(red+blue)
G.add_nodes_from(nodesList)
makeGraph(sys.path[0] + graphFile)
# G = nx.convert_node_labels_to_integers(G)

outRB()
lR = initlR()



P = nx.adjacency_matrix(G)
P = P.toarray()
row_sums = P.sum(axis=1)
P = (P / row_sums[:, np.newaxis])

where_are_NaNs = np.isnan(P)
P[where_are_NaNs] = 0

u = np.empty(len(P)) 
u.fill(1/len(P))
u = np.transpose(u)

# Handle sink nodes
d = np.zeros(len(P))

for node in G.nodes:
    if G.out_degree(node) == 0:
        d[node] = 1
P = P + np.outer(d, u)

I = np.identity(len(P))
D = np.zeros(shape=(len(P), len(P)))

# Random jump vector
v = np.empty(len(P)) 
v.fill(1/len(G.nodes))

# Vector with 1 over red nodes
color = np.zeros(len(P))
for node in red:
    color[node] = 1
    
gamma = np.zeros(len(P)) 
for node in G.nodes:
    if node in red:
        gamma[node] = gRed
    else:
        gamma[node] = gBlue
gamma = np.diag(gamma)

# intrinsic opinions
s = np.zeros(len(P))
for node in G.nodes:
    if node in red:
        s[node] = 1
    else:
        s[node] = -1

In [10]:
Q = (np.linalg.inv(I-((I-gamma).dot(P)))).dot(gamma)
z = Q @ s

In [11]:
start_time = time.time()
results = defaultdict(lambda: {"Fairness": 0.0, "CostV": 0.0, "CostS": 0.0})

fair = np.zeros(len(P)) 

fairNodes = []
for fNodes in (range(0, len(lR) + 1)):
    print(fNodes, "fair nodes are: ", fairNodes[:fNodes])
    Q = (np.linalg.inv(I-((I-gamma).dot(P+D)))).dot(gamma)
    zPrime = Q @ s
    c = (zPrime - z) ** 2
    results[fNodes]["CostV"] = np.sum(c)/len(G.nodes)
    results[fNodes]["CostS"] = np.sum(c*fair) / max(1, fNodes)
    pr = v.dot(Q)
    prRed = np.inner(pr, color)
    print("PageRank allocated to red group: ", prRed)
    if len(lR) > 0:
        gains = cmpFormula()
        nFair = max(gains, key=gains.get)
        print(nFair, gains[nFair], "-----")
        lR.remove(nFair)
        fairNodes.append(nFair)
        updateD(nFair)
        fair[nFair] = 1
    results[fNodes]["Fairness"] = prRed
print("--- %s seconds ---" % (time.time() - start_time))

0 fair nodes are:  []
PageRank allocated to red group:  0.49695722518163954
23 0.12226416335517624 -----
1 fair nodes are:  [23]
PageRank allocated to red group:  0.6192213885368159
25 0.039054544513706255 -----
2 fair nodes are:  [23, 25]
PageRank allocated to red group:  0.658275933050522
21 0.02528341940985163 -----
3 fair nodes are:  [23, 25, 21]
PageRank allocated to red group:  0.6835593524603736
32 0.012179726917422888 -----
4 fair nodes are:  [23, 25, 21, 32]
PageRank allocated to red group:  0.6957390793777964
13 0.009215062429439326 -----
5 fair nodes are:  [23, 25, 21, 32, 13]
PageRank allocated to red group:  0.7049541418072358
22 0.0062594625809185974 -----
6 fair nodes are:  [23, 25, 21, 32, 13, 22]
PageRank allocated to red group:  0.7112136043881544
28 0.0035496608093723097 -----
7 fair nodes are:  [23, 25, 21, 32, 13, 22, 28]
PageRank allocated to red group:  0.7147632651975269
2 0.0023601627948513708 -----
8 fair nodes are:  [23, 25, 21, 32, 13, 22, 28, 2]
PageRank al